In [1]:
from os import environ
environ['OMP_NUM_THREADS'] = '12'
import numpy as np
import scipy.linalg as la

In [2]:
qubit_num = 5 #число кубитов для вычисления собственного значения
unitary_eigs = np.repeat(
    a=[la.expm(1j*2*np.pi*i/(2**qubit_num)) for i in range(2**qubit_num)],
    repeats=1)
hermitian_eigs = [np.log(i) for i in unitary_eigs]
A = np.diag(hermitian_eigs).astype('complex')
U = la.expm(1*A)
eigs_u, vecs_u = np.linalg.eig(U)

In [3]:
#states
state_zero = np.array([[1.0],[0.0]])
state_one = np.array([[0.0],[1.0]])
#projectors
P0 = np.dot(state_zero, state_zero.T)
P1 = np.dot(state_one, state_one.T)

In [4]:
def multi_kron(*args):
    ret = np.array([[1.0]]).astype('complex')
    for q in args: 
        ret = np.kron(ret, q)
    return ret

def multi_dot(*args):
    ret = np.eye(np.shape(args[0])[0]).astype('complex')
    for q in args:
        ret = np.dot(ret, q)
    return ret

In [5]:
def H():
    return 1/np.sqrt(2) * np.array([[1, 1],
                                    [1, -1]]).astype('complex')
def X():
    return np.array([[0, 1],
                     [1, 0]]).astype('complex')
def Rz(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]]).astype('complex')
def P(phi):
    return np.array([[1, 0],
                     [0, np.exp(1j*phi)]]).astype('complex')
def Ry(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]]).astype('complex')
def I():
    return np.array([[1, 0],
                     [0, 1]]).astype('complex')

#for some reson CONTROL and TARGET are changed
def control(control_q, target_q, n_of_qubits, operator):
    list_of_ops_left = []
    list_of_ops_right = []
    stop = False
    for i in range(n_of_qubits):
        if i in control_q:
            list_of_ops_left.append(P0)
            list_of_ops_right.append(P1)
        elif i in target_q:
            if np.shape(U) != (2, 2):
                if not stop:
                    for _ in range(len(target_q)):
                        list_of_ops_left.append(I())
                    list_of_ops_right.append(operator)
                    stop = True
            else:
                list_of_ops_left.append(I())
                list_of_ops_right.append(operator)
        else:
            list_of_ops_left.append(I())
            list_of_ops_right.append(I())
    return multi_kron(*list_of_ops_left) + multi_kron(*list_of_ops_right).astype('complex')

In [6]:
#eigenvalues to phases
def eigenvalues_to_phases(n, u):
    repetitions = 1
    ops = []
    target_qubits_amount = int(np.log2(u.shape[0]))
    for i in range(0, n):
        for _ in range(repetitions):
            ops.append(control(control_q=[target_qubits_amount-1+n-i],
                               target_q=[idx for idx in range(target_qubits_amount)],
                               n_of_qubits=n+target_qubits_amount,
                               operator=u**1
                               ))
        repetitions *= 2
    return multi_dot(*ops)

In [7]:
from qiskit import QuantumCircuit, Aer, execute
def swap(n_of_qubs,u):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    target_qubits_amount = int(np.log2(u.shape[0]))
    qc = QuantumCircuit(n_of_qubs+target_qubits_amount)
    for qubit in range(n_of_qubs//2): #n//2
        qc.swap(qubit, n_of_qubs-qubit-1)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    op = result.get_unitary(qc).data
    return op

def iqft(n, u):
    ops = []
    target_qubits_amount = int(np.log2(u.shape[1]))
    hadamards = np.array([I() for _ in range(n+target_qubits_amount)])
    for j in range(n): 
        for m in range(j):
            ops.append(
                control(
                control_q=[target_qubits_amount - 1 + n - m],
                target_q=[target_qubits_amount - 1 + n - j],
                n_of_qubits=n + target_qubits_amount,
                operator=P(-np.pi/float(2**(j-m))),
                )
            )
        hadamards_ = hadamards.copy()
        hadamards_[target_qubits_amount - 1 + n - j] = H()
        ops.append(multi_kron(*hadamards_))
    return multi_dot(*ops)

In [8]:
prime_state = multi_kron(
    np.array([[el] for el in vecs_u[0]]),
    *[state_zero for _ in range(qubit_num)])
#state preparation
state_prep = multi_kron(*[I() for _ in range(int(np.log2(len(vecs_u[0]))))],
                        *[H() for _ in range(qubit_num)])
#calculations
eigs_to_phases = eigenvalues_to_phases(qubit_num, U)
iqft_matrix = iqft(qubit_num, U)
ift = multi_dot(swap(qubit_num, U), iqft_matrix)
final_u = multi_dot(ift, eigs_to_phases, state_prep)
final_state = np.dot(final_u, prime_state)
rho = np.dot(final_state, np.conj(final_state).T)
# vecs_u

In [9]:
max_num = 2**qubit_num
max_prob = 0
eigenvalue = -9999
probs = []
for i in range(max_num):
    ops = [I() for _ in range(int(np.log2(len(vecs_u[0]))))]
    bin_int = "{0:b}".format(i).zfill(qubit_num)
    ops.extend([P0 if bit == '0' else P1 for bit in bin_int])
    proj = multi_kron(*ops)
    prob = np.trace(np.dot(proj, rho))
    probs.append(np.round(prob,3))
    if prob > max_prob:
        max_prob = prob
        eigenvalue = la.expm(1j*2*np.pi*i/(2**qubit_num))
print(f'calculated eigenvalue is {np.round(eigenvalue, 3)[0]}')
print(f'real eigenvalus are {np.round(eigs_u, 3)}')
print(f'all probabilitis {probs}')

calculated eigenvalue is [1.+0.j]
real eigenvalus are [ 1.   +0.j     0.981+0.195j  0.924+0.383j  0.831+0.556j  0.707+0.707j
  0.556+0.831j  0.383+0.924j  0.195+0.981j  0.   +1.j    -0.195+0.981j
 -0.383+0.924j -0.556+0.831j -0.707+0.707j -0.831+0.556j -0.924+0.383j
 -0.981+0.195j -1.   +0.j    -0.981-0.195j -0.924-0.383j -0.831-0.556j
 -0.707-0.707j -0.556-0.831j -0.383-0.924j -0.195-0.981j -0.   -1.j
  0.195-0.981j  0.383-0.924j  0.556-0.831j  0.707-0.707j  0.831-0.556j
  0.924-0.383j  0.981-0.195j]
all probabilitis [(1+0j), -0j, -0j, 0j, -0j, 0j, -0j, -0j, 0j, 0j, 0j, -0j, -0j, -0j, -0j, -0j, 0j, -0j, -0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, -0j, -0j, -0j, -0j, 0j]
